<a href="https://colab.research.google.com/github/marco-siino/llm-network-testbed/blob/main/blueprint/simulated-environment/codeGemma/1-shot-prompt/CodeGemma_7B_IT_BluePrintToNS3_1_shot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing libraries and load the model.

In [1]:
import os
import random
import torch

from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer

model_id = "cache/models--google--codegemma-7b-it/snapshots/078cdc51070553d1636d645c9a238f3b0914459a/"

#model_id = "google/gemma-7b-it"
dtype = torch.bfloat16

# I can decide which GPU to use on this node on Leonardo.
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"] = "0"
torch.cuda.set_device(0)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=dtype,
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [2]:
!nvidia-smi

Wed Dec  4 15:52:53 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 530.30.02    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM-64GB            On | 00000000:1D:00.0 Off |                    0 |
| N/A   44C    P0               75W / 459W|  14319MiB / 65536MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

# Generate the NS3 code for the smart_agriculture scenario.

In [3]:
with open("projects/testbed-at-JNCA/blueprint/sample_blueprint.json") as f:
        sample_blueprint = f.read()
with open("projects/testbed-at-JNCA/blueprint/sample_ns3_code.cc") as f:
        sample_ns3_code = f.read()
with open("projects/testbed-at-JNCA/blueprint/smart_agriculture.json") as f:
        test_blueprint = f.read()

In [4]:
prompt = """
 Given the following SAMPLE BLUEPRINT and the corresponding SAMPLE NS3 code, use them as a sample to generate the NS3 code for the TEST BLUEPRINT:

        SAMPLE BLUEPRINT:

        """+sample_blueprint+ """

        SAMPLE NS3 CODE:

        """+sample_ns3_code+"""

        TEST BLUEPRINT:

        """+test_blueprint+"""

        TEST NS3 CODE:

        """
print(prompt)


 Given the following SAMPLE BLUEPRINT and the corresponding SAMPLE NS3 code, use them as a sample to generate the NS3 code for the TEST BLUEPRINT:

        SAMPLE BLUEPRINT:

        {
  "name": "Smart Home",
  "description": "A network blueprint for a smart home with various automated devices.",
  "components": [

    {"type": "WiFi AP",
      "id": "AP-001",
      "application": "WiFi Access Point",
      "location": "City Hall",      
      "protocol": "WiFi",
      "ip_address":"192.168.1.1",        
      "latitude": 38.10351066811091,
      "longitude": 13.3459399220741
    },
    {
      "type": "Smart Thermostat",
      "id": "ST-001",
      "location": "Hallway",
      "temperature_range": "18-25°C",
      "energy_efficiency": "A+" ,     
      "protocol":"Wi-Fi",
      "latitude": 38.10863528672426,
      "longitude": 13.34050633101243,
      "ip_address":"192.168.1.2"
    },
    {
      "type": "Smart Light Bulb",
      "id": "SLB-001",
      "location": "Kitchen",
      "b

In [5]:
input_ids = tokenizer(prompt, return_tensors="pt")

outputs = model.generate(**input_ids, max_new_tokens=5500)
print(tokenizer.decode(outputs[0]))

/leonardo/home/userexternal/msiino00/clean-updated-env/lib/python3.10/site-packages/transformers/generation/utils.py:2097: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (8192). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


<bos>
 Given the following SAMPLE BLUEPRINT and the corresponding SAMPLE NS3 code, use them as a sample to generate the NS3 code for the TEST BLUEPRINT:

        SAMPLE BLUEPRINT:

        {
  "name": "Smart Home",
  "description": "A network blueprint for a smart home with various automated devices.",
  "components": [

    {"type": "WiFi AP",
      "id": "AP-001",
      "application": "WiFi Access Point",
      "location": "City Hall",      
      "protocol": "WiFi",
      "ip_address":"192.168.1.1",        
      "latitude": 38.10351066811091,
      "longitude": 13.3459399220741
    },
    {
      "type": "Smart Thermostat",
      "id": "ST-001",
      "location": "Hallway",
      "temperature_range": "18-25°C",
      "energy_efficiency": "A+" ,     
      "protocol":"Wi-Fi",
      "latitude": 38.10863528672426,
      "longitude": 13.34050633101243,
      "ip_address":"192.168.1.2"
    },
    {
      "type": "Smart Light Bulb",
      "id": "SLB-001",
      "location": "Kitchen",
   

## Save the .cc code into a file.

In [6]:
s = tokenizer.decode(outputs[0])
s = s.split("TEST NS3 CODE:", 1)[-1].lstrip().rstrip()

with open("projects/testbed-at-JNCA/blueprint/codegemma/smart_agriculture-1_shot.cc", "w") as file:
    file.write(s)

# Generate the NS3 code for the smart_city scenario.

In [7]:
with open("projects/testbed-at-JNCA/blueprint/smart_city.json") as f:
        test_blueprint = f.read()

In [8]:
prompt = """
 Given the following SAMPLE BLUEPRINT and the corresponding SAMPLE NS3 code, use them as a sample to generate the NS3 code for the TEST BLUEPRINT:

        SAMPLE BLUEPRINT:

        """+sample_blueprint+ """

        SAMPLE NS3 CODE:

        """+sample_ns3_code+"""

        TEST BLUEPRINT:

        """+test_blueprint+"""

        TEST NS3 CODE:

        """
print(prompt)


 Given the following SAMPLE BLUEPRINT and the corresponding SAMPLE NS3 code, use them as a sample to generate the NS3 code for the TEST BLUEPRINT:

        SAMPLE BLUEPRINT:

        {
  "name": "Smart Home",
  "description": "A network blueprint for a smart home with various automated devices.",
  "components": [

    {"type": "WiFi AP",
      "id": "AP-001",
      "application": "WiFi Access Point",
      "location": "City Hall",      
      "protocol": "WiFi",
      "ip_address":"192.168.1.1",        
      "latitude": 38.10351066811091,
      "longitude": 13.3459399220741
    },
    {
      "type": "Smart Thermostat",
      "id": "ST-001",
      "location": "Hallway",
      "temperature_range": "18-25°C",
      "energy_efficiency": "A+" ,     
      "protocol":"Wi-Fi",
      "latitude": 38.10863528672426,
      "longitude": 13.34050633101243,
      "ip_address":"192.168.1.2"
    },
    {
      "type": "Smart Light Bulb",
      "id": "SLB-001",
      "location": "Kitchen",
      "b

In [9]:
input_ids = tokenizer(prompt, return_tensors="pt")

outputs = model.generate(**input_ids, max_new_tokens=5500)
print(tokenizer.decode(outputs[0]))

<bos>
 Given the following SAMPLE BLUEPRINT and the corresponding SAMPLE NS3 code, use them as a sample to generate the NS3 code for the TEST BLUEPRINT:

        SAMPLE BLUEPRINT:

        {
  "name": "Smart Home",
  "description": "A network blueprint for a smart home with various automated devices.",
  "components": [

    {"type": "WiFi AP",
      "id": "AP-001",
      "application": "WiFi Access Point",
      "location": "City Hall",      
      "protocol": "WiFi",
      "ip_address":"192.168.1.1",        
      "latitude": 38.10351066811091,
      "longitude": 13.3459399220741
    },
    {
      "type": "Smart Thermostat",
      "id": "ST-001",
      "location": "Hallway",
      "temperature_range": "18-25°C",
      "energy_efficiency": "A+" ,     
      "protocol":"Wi-Fi",
      "latitude": 38.10863528672426,
      "longitude": 13.34050633101243,
      "ip_address":"192.168.1.2"
    },
    {
      "type": "Smart Light Bulb",
      "id": "SLB-001",
      "location": "Kitchen",
   

## Save the .cc code into a file.

In [10]:
s = tokenizer.decode(outputs[0])
s = s.split("TEST NS3 CODE:", 1)[-1].lstrip().rstrip()

with open("projects/testbed-at-JNCA/blueprint/codegemma/smart_city-1_shot.cc", "w") as file:
    file.write(s)

# Generate the NS3 code for the smart_home scenario.

In [11]:
with open("projects/testbed-at-JNCA/blueprint/smart_home.json") as f:
        test_blueprint = f.read()

In [14]:
prompt = """
 Given the following SAMPLE BLUEPRINT and the corresponding SAMPLE NS3 code, use them as a sample to generate the NS3 code for the TEST BLUEPRINT:

        SAMPLE BLUEPRINT:

        """+sample_blueprint+ """

        SAMPLE NS3 CODE:

        """+sample_ns3_code+"""

        TEST BLUEPRINT:

        """+test_blueprint+"""

        TEST NS3 CODE:

        """
print(prompt)


 Given the following SAMPLE BLUEPRINT and the corresponding SAMPLE NS3 code, use them as a sample to generate the NS3 code for the TEST BLUEPRINT:

        SAMPLE BLUEPRINT:

        {
  "name": "Smart Home",
  "description": "A network blueprint for a smart home with various automated devices.",
  "components": [

    {"type": "WiFi AP",
      "id": "AP-001",
      "application": "WiFi Access Point",
      "location": "City Hall",      
      "protocol": "WiFi",
      "ip_address":"192.168.1.1",        
      "latitude": 38.10351066811091,
      "longitude": 13.3459399220741
    },
    {
      "type": "Smart Thermostat",
      "id": "ST-001",
      "location": "Hallway",
      "temperature_range": "18-25°C",
      "energy_efficiency": "A+" ,     
      "protocol":"Wi-Fi",
      "latitude": 38.10863528672426,
      "longitude": 13.34050633101243,
      "ip_address":"192.168.1.2"
    },
    {
      "type": "Smart Light Bulb",
      "id": "SLB-001",
      "location": "Kitchen",
      "b

In [15]:
input_ids = tokenizer(prompt, return_tensors="pt")

outputs = model.generate(**input_ids, max_new_tokens=5500)
print(tokenizer.decode(outputs[0]))

<bos>
 Given the following SAMPLE BLUEPRINT and the corresponding SAMPLE NS3 code, use them as a sample to generate the NS3 code for the TEST BLUEPRINT:

        SAMPLE BLUEPRINT:

        {
  "name": "Smart Home",
  "description": "A network blueprint for a smart home with various automated devices.",
  "components": [

    {"type": "WiFi AP",
      "id": "AP-001",
      "application": "WiFi Access Point",
      "location": "City Hall",      
      "protocol": "WiFi",
      "ip_address":"192.168.1.1",        
      "latitude": 38.10351066811091,
      "longitude": 13.3459399220741
    },
    {
      "type": "Smart Thermostat",
      "id": "ST-001",
      "location": "Hallway",
      "temperature_range": "18-25°C",
      "energy_efficiency": "A+" ,     
      "protocol":"Wi-Fi",
      "latitude": 38.10863528672426,
      "longitude": 13.34050633101243,
      "ip_address":"192.168.1.2"
    },
    {
      "type": "Smart Light Bulb",
      "id": "SLB-001",
      "location": "Kitchen",
   

## Save the .cc code into a file.

In [17]:
s = tokenizer.decode(outputs[0])
s = s.split("TEST NS3 CODE:", 1)[-1].lstrip().rstrip()

with open("projects/testbed-at-JNCA/blueprint/codegemma/smart_home-1_shot.cc", "w") as file:
    file.write(s)